## Physics Toolbox Sensor Suite 6軸IMU數據切割程式

### 1.匯入函式庫

In [ ]:
#繪圖
from bokeh.models import Span
from bokeh.plotting import figure, output_file, save
#數據分析
import pandas as pd
#資料搜索
import glob
import os
#建立Excel檔
from pyexcelerate import Workbook
#建立資料夾
from pathlib import Path
#建立壓縮檔
from zipfile import ZipFile
#獲得本日時間
from datetime import date

### 2.宣告輸出資料夾及不需要處理的檔案

In [ ]:
today = date.today()
o_dir = "./" + today.strftime("%Y%m%d") + "/" #預設為本日時間，可自行設定，ex: o_dir = "./xxx/"
skip_file = [] #跳過不需要處理的csv檔，ex: skip_file = ["xxx.csv"]

### 3.1 定義切割數據函式

In [ ]:
def cut_data(data_df):
    fragment = []
    silence = False
    data_filtered = (data_df["ax"] < 0.3) & (data_df["ax"] > -0.3)
    cnt = 0
    for idx in range(len(data_filtered)):
        if data_filtered[idx]:
            cnt = cnt + 1
            wave = 0
        else:
            wave = wave + 1
            if wave > 15:
                cnt = 0
                if silence:
                    fragment.append(idx - 1)
                    silence = False
        if cnt > 300:
            silence = True
            if idx == len(data_filtered)-1:
                fragment.append(idx)
    silence = False
    data_filtered = (data_df["ax"] < 0.3) & (data_df["ax"] > -0.3)
    cnt = 0
    for idx in reversed(range(len(data_filtered))):
        if data_filtered[idx]:
            cnt = cnt + 1
            wave = 0
        else:
            wave = wave + 1
            if wave > 15:
                cnt = 0
                if silence:
                    fragment.append(idx + 1)
                    silence = False
        if cnt > 300:
            silence = True
            if idx == 0:
                fragment.append(idx)
    fragment.sort()
    return fragment

### 3.2定義顯示波型函式

In [ ]:
def write_image_full(data_df, fragment, write_path):
    p = figure(
        title="origin",
        x_axis_label="time",
        y_axis_label="ax",
        plot_width=1500,
        plot_height=800,
    )
    p.line(data_df.time, data_df.ax, legend_label="Amplitude", line_width=2)
    for idx in fragment:
        line = data_df.time[idx]
        vline = Span(
            location=line,
            dimension="height",
            line_color="red",
            line_dash="dashed",
            line_width=2,
        )
        p.add_layout(vline)
    output_file(write_path)
    save(p)

In [ ]:
def write_image_part(data_df, fragment, idx, write_path):
    p = figure(
        title="origin",
        x_axis_label="time",
        y_axis_label="ax",
        plot_width=1500,
        plot_height=800,
    )
    start = fragment[idx]
    end = fragment[idx + 3]
    p.line(data_df.time[start:end], data_df.ax[start:end], legend_label="Amplitude", line_width=2)
    output_file(write_path)
    save(p)

### 4.切割資料、輸出excel檔及壓縮成zip檔

In [ ]:
print("[Create folder] " + o_dir)
print("-"*60)
Path(o_dir).mkdir(parents=True, exist_ok=True)
for files in glob.glob('*.csv'):
    print("[Create folder] " + o_dir + files[:-4])
    Path(o_dir + files[:-4]).mkdir(parents=True, exist_ok=True)
    data = pd.read_csv(files).drop(columns=['Unnamed: 7'])
    file_cnt = 0
    wb = Workbook()
    if files in skip_file:
        write_image_full(data, fragment, o_dir + files[:-4] + "_full.html")
        print("[Write] " + o_dir + files[:-4] + ".xlsx, sheet_name:" + "raw")
        output_data = [
                data.columns.tolist(),
            ] + data.values.tolist()
        wb.new_sheet("raw", data=output_data)
        wb.save(o_dir + files[:-4] + ".xlsx")
        print("-"*60)
    else:
        fragment = cut_data(data)
        write_image_full(data, fragment, o_dir + files[:-4] + "_full.html")
        for idx in range(0, len(fragment) - 2, 2):
            file_cnt = file_cnt + 1
            write_image_part(data, fragment, idx, o_dir + files[:-4] +"/part_" + str(file_cnt) + ".html")
            print("[Write] " + o_dir + files[:-4] + ".xlsx, sheet_name:" + "part_" + str(file_cnt))
            file_idx_s = fragment[idx]
            file_idx_e = fragment[idx + 3]
            output_data = [
                data[file_idx_s:file_idx_e].columns.tolist(),
            ] + data[file_idx_s:file_idx_e].values.tolist()
            wb.new_sheet("raw_" + str(file_cnt), data=output_data)
            wb.save(o_dir + files[:-4] + ".xlsx")
        print("-"*60)
print("[Compression folder] " + o_dir[:-1])
with ZipFile(o_dir[:-1] + ".zip", 'w') as zfile:
    for root, dirs, files in os.walk(o_dir):
        for file in files:
            zfile.write(os.path.join(root, file))
print("-"*60)
print("[Done]")